# Аудиториска вежба 3

In [1]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

## Examples

In [5]:
p = pipeline(task='text-generation', model='gpt2')

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
sentence = 'Hello, I\'m a language model'

In [9]:
result1 = p(sentence)
print(result1[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, I'm a language model in Scala. I didn't set the number of symbols to 64 characters when writing the Haskell code but I can make sure that any number of bits are allowed. You can make multiple copies of the variable, any one


In [10]:
result2 = p(sentence, return_full_text=False)
print(result2[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


, it's hard to express, there's this other way to express something. And I use it, I think this, but I think the whole design thing, it's not easy to explain to people that you


In [11]:
result3 = p(sentence, return_full_text=False, max_length=25)
print(result3[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


. This is based off of the fact that I'm a software architect. It's based


## Zero-shot

In [12]:
data = pd.read_csv('train_en.txt', sep='\t')

In [66]:
sentences = data['Sentence'].values.tolist()
labels = data['Style'].values.tolist()

In [14]:
model_name = 't5-base'

In [15]:
t5_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:
t5_tokenizer = AutoTokenizer.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
sample_sentence = sentences[3]
print(sample_sentence)

the garbage that is being created by cnn and other news agencies is outrageous .


Classify the following sentence into toxic or neutral: "text"

In [74]:
tokens = t5_tokenizer(f'Classify the following sentence into toxic or neutral: {sample_sentence}', return_tensors='pt')

In [75]:
tokens

{'input_ids': tensor([[ 4501,  4921,     8,   826,  7142,   139, 12068,    42,  7163,    10,
             8, 12937,    24,    19,   271,   990,    57,     3,    75,    29,
            29,    11,   119,  1506,  4299,    19, 30662,     3,     5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]])}

In [76]:
predictions = t5_model.generate(tokens.input_ids)
predictions

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    0, 32099,    10, 12068,    42,  7163,    10,     8, 12937,    24,
            19,   271,   990,    57,     3,    75,    29,    29,    11,   119]])

In [77]:
t5_tokenizer.decode(predictions[0])

'<pad><extra_id_0>: toxic or neutral: the garbage that is being created by cnn and other'

## Few-shot

In [78]:
examples = f'stop the coverage and let em rot . -> toxic \n stop the coverage and let em rot . -> neutral'

In [79]:
examples

'stop the coverage and let em rot . -> toxic \n stop the coverage and let em rot . -> neutral'

In [80]:
prompt = f'{examples}\n Classify the following sentence into toxic or neutral: {sample_sentence}'

In [81]:
prompt

'stop the coverage and let em rot . -> toxic \n stop the coverage and let em rot . -> neutral\n Classify the following sentence into toxic or neutral: the garbage that is being created by cnn and other news agencies is outrageous .'

In [82]:
tokens_2 = t5_tokenizer(prompt, return_tensors='pt')

In [83]:
tokens_2

{'input_ids': tensor([[ 1190,     8,  2591,    11,   752,     3,    15,    51,     3,  2719,
             3,     5,     3, 13114, 12068,  1190,     8,  2591,    11,   752,
             3,    15,    51,     3,  2719,     3,     5,     3, 13114,  7163,
          4501,  4921,     8,   826,  7142,   139, 12068,    42,  7163,    10,
             8, 12937,    24,    19,   271,   990,    57,     3,    75,    29,
            29,    11,   119,  1506,  4299,    19, 30662,     3,     5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [85]:
predictions_2 = t5_model.generate(tokens_2.input_ids)
predictions_2

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    0,     3, 13114,  7163,  1190,     8,  2591,    11,   752,     3,
            15,    51,     3,  2719,     3,     5,     3, 13114, 12068,  1190]])

In [86]:
t5_tokenizer.decode(predictions_2[0])

'<pad> -> neutral stop the coverage and let em rot. -> toxic stop'

In [49]:
embedding_model = SentenceTransformer('all-distilroberta-v1')
embeddings = embedding_model.encode(sentences[:20], batch_size=64, show_progress_bar=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
embeddings

array([[ 0.05055266, -0.06140864, -0.01228615, ..., -0.03881474,
        -0.06116225,  0.01812133],
       [-0.00400063, -0.0094031 , -0.00860852, ..., -0.06624115,
        -0.04656709, -0.02007009],
       [ 0.02266759, -0.02643988,  0.00947504, ..., -0.0447992 ,
        -0.05901451,  0.01749855],
       ...,
       [-0.02219027, -0.04576226,  0.02153145, ...,  0.04361862,
         0.01497844,  0.04335181],
       [ 0.06613795, -0.06282373, -0.03906499, ..., -0.06832648,
        -0.0715312 ,  0.07797877],
       [ 0.04563303, -0.01121642, -0.01660859, ..., -0.04931673,
        -0.05503172,  0.07557258]], dtype=float32)

In [51]:
sample_sentence_emb = embedding_model.encode(sample_sentence, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [52]:
sample_sentence_emb

array([ 1.02316178e-02, -6.45948574e-02,  5.73721044e-02, -6.93059936e-02,
        4.48392928e-02, -1.61739271e-02,  2.07158132e-03, -1.90986749e-02,
        1.78208798e-02, -5.58549128e-02, -1.84251089e-02, -1.23199755e-02,
       -4.46541831e-02, -8.49483013e-02, -4.33981344e-02, -7.88730755e-02,
       -4.21902873e-02, -3.32277524e-03,  2.49044411e-02,  4.51897644e-02,
        1.49633726e-02,  1.54996710e-02, -5.28334603e-02,  5.79168042e-03,
        3.30365896e-02, -4.50554751e-02,  4.21211608e-02, -1.11803710e-02,
       -6.08209297e-02,  1.89126353e-03, -3.80693451e-02,  3.50914821e-02,
       -8.14382359e-03,  3.66329327e-02, -2.50468217e-02, -2.36641453e-03,
        4.56128381e-02,  2.15694830e-02,  3.67684178e-02, -2.99004409e-02,
        3.23758721e-02,  1.02443948e-01,  3.20526597e-04,  1.72446482e-02,
        5.30676693e-02,  1.44669320e-05, -9.57474932e-02,  1.64969340e-02,
        2.00691707e-02,  8.54183640e-03, -1.23227341e-02, -3.11375072e-04,
        8.79212469e-03, -

In [59]:
most_similar = util.semantic_search(sample_sentence_emb, embeddings, top_k=4)
most_similar

[[{'corpus_id': 3, 'score': 0.9999999403953552},
  {'corpus_id': 7, 'score': 0.35389444231987},
  {'corpus_id': 17, 'score': 0.33577677607536316},
  {'corpus_id': 19, 'score': 0.3139081597328186}]]

In [57]:
sample_sentence

'the garbage that is being created by cnn and other news agencies is outrageous .'

In [62]:
ex_1 = most_similar[0][1]['corpus_id']
ex_1

7

In [64]:
ex_1_text = sentences[ex_1]
ex_1_text

'stop the coverage and let em rot .'

In [67]:
ex_1_label = labels[ex_1]
ex_1_label

'toxic'

In [68]:
example_1 = f'{ex_1_text} -> {ex_1_label}'
example_1

'stop the coverage and let em rot . -> toxic'

In [69]:
f'{ex_1_text}: {ex_1_label}'

'stop the coverage and let em rot .: toxic'

In [72]:
f'Sentence: {ex_1_text}. Label: {ex_1_label}'

'Sentence: stop the coverage and let em rot .. Label: toxic'

In [73]:
f'{example_1}\n{sample_sentence}'

'stop the coverage and let em rot . -> toxic\nthe garbage that is being created by cnn and other news agencies is outrageous .'

In [71]:
sentences[7]

'stop the coverage and let em rot .'